In [1]:
import os
os.getcwd()

'/Users/yunhan/Documents/ForHanhan/DTS/dts_mlops2015/mlops-zoomcamps/02-experiment-tracking'

In [2]:
import pandas as pd
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [18]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [5]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

/Users/yunhan/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
2025/05/25 16:51:13 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/yunhan/Documents/ForHanhan/DTS/dts_mlops2015/mlops-zoomcamps/02-experiment-tracking/mlruns/1', creation_time=1748184673490, experiment_id='1', last_update_time=1748184673490, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
df_train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,PU_DO
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00,3.933333,43_151
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75,8.750000,166_239
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00,5.966667,41_42
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00,7.083333,168_75
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.5,...,0.96,0.0,None,0.3,5.76,1.0,1.0,0.00,2.316667,75_75


In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
print(type(y_val), type(y_pred))

root_mean_squared_error(y_val, y_pred) # since scipy 1.6, mean_squared_error delete variable squared

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


7.758715207732648

In [20]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [21]:
with mlflow.start_run():

    mlflow.set_tag("developer", "yunhan")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [22]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [23]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [24]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [27]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.70964                           
[1]	validation-rmse:8.30383                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/yunhan/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:51:07] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:7.55213                           
[3]	validation-rmse:7.15429                           
[4]	validation-rmse:6.94449                           
[5]	validation-rmse:6.83035                           
[6]	validation-rmse:6.77022                           
[7]	validation-rmse:6.73338                           
[8]	validation-rmse:6.70931                           
[9]	validation-rmse:6.69371                           
[10]	validation-rmse:6.68474                          
[11]	validation-rmse:6.67862                          
[12]	validation-rmse:6.67571                          
[13]	validation-rmse:6.67069                          
[14]	validation-rmse:6.66716                          
[15]	validation-rmse:6.66357                          
[16]	validation-rmse:6.66051                          
[17]	validation-rmse:6.65635                          
[18]	validation-rmse:6.65147                          
[19]	validation-rmse:6.64929                          
[20]	valid

/Users/yunhan/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:52:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.64092                                                   
[1]	validation-rmse:11.11995                                                   
[2]	validation-rmse:10.64705                                                   
[3]	validation-rmse:10.21767                                                   
[4]	validation-rmse:9.83005                                                    
[5]	validation-rmse:9.47914                                                    
[6]	validation-rmse:9.16380                                                    
[7]	validation-rmse:8.88037                                                    
[8]	validation-rmse:8.62343                                                    
[9]	validation-rmse:8.39471                                                    
[10]	validation-rmse:8.18912                                                   
[11]	validation-rmse:8.00571                                                   
[12]	validation-rmse:7.84189            

...

[1]	validation-rmse:6.74818                                                     
[2]	validation-rmse:6.73035                                                     
[3]	validation-rmse:6.71730                                                     
[4]	validation-rmse:6.71292                                                     
[5]	validation-rmse:6.71026                                                     
[6]	validation-rmse:6.70610                                                     
[7]	validation-rmse:6.69707                                                     
[8]	validation-rmse:6.69204                                                     
[9]	validation-rmse:6.68938                                                     
[10]	validation-rmse:6.68699                                                    
[11]	validation-rmse:6.68269                                                    
[12]	validation-rmse:6.67871                                                    
[13]	validation-rmse:6.67470

/Users/yunhan/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:24:13] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[6]	validation-rmse:6.75590                                                     
[7]	validation-rmse:6.74806                                                     
[8]	validation-rmse:6.74311                                                     
[9]	validation-rmse:6.74187                                                     
[10]	validation-rmse:6.73770                                                    
[11]	validation-rmse:6.72590                                                    
[12]	validation-rmse:6.72115                                                    
[13]	validation-rmse:6.71959                                                    
[14]	validation-rmse:6.71317                                                    
[15]	validation-rmse:6.71359                                                    
[16]	validation-rmse:6.71170                                                    
[17]	validation-rmse:6.70871                                                    
[18]	validation-rmse:6.70722

/Users/yunhan/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:24:33] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:7.05600                                                     
[3]	validation-rmse:6.87246                                                     
[4]	validation-rmse:6.78704                                                     
[5]	validation-rmse:6.74520                                                     
[6]	validation-rmse:6.72146                                                     
[7]	validation-rmse:6.70883                                                     
[8]	validation-rmse:6.69818                                                     
[9]	validation-rmse:6.69028                                                     
[10]	validation-rmse:6.68864                                                    
[11]	validation-rmse:6.68734                                                    
[12]	validation-rmse:6.68523                                                    
[13]	validation-rmse:6.68376                                                    
[14]	validation-rmse:6.68164

In [26]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'